# Spatial Computing and Data Mining to Predict Restaurent Succeess

## Introduction

1. The number one factor for the success of a restaurant is the location and population density and demography. The study conducted by Dr. H.G. Parsa, [1] proves this statement by analyzing restaurant failures in Boulder, Colorado. 

2. But there has not been done any research with a big-data set such as Yelp academic dataset to analyze several cities with thousands of businesses and other related data which can contribute to restaurant success of failures.

3. In this analysis we will investigate questions such as, how much of the restaurant success is accountable to its location, how urban and City planners can be benefited from this type of study or lenders or banks can assess restaurant risk factors and predict future success, how types of restaurants distributed in a city and what type of restaurant has high tendancy to success in a particular area. 

4. The study will be devided in to several sectoin based on the tasks needed to correctly analyse the spatial dependance of restaurants in the dataset. To measure the success of a restaurant, review_count provided in the business dataset is used. I did not use the star rating as a measure because it may not be a fair measure ,becasue there will not be a way to distinguish between rate of change of the review_count is considerd as a meauser. so only count the rewviews in 2014. Yelp is popular among many peopple by 2014 it is better to count how many review_counts happend in 2014. 

## Outline of the notebook
  
- Predict success of a restarurant based on location related predictor variables
    - Feature engineering
    - Spatial Autocorrelation Analsys
    - Spatial Autoregressoin Model Building
    - Modal Evaluation

- Hot-Spot Analysis
- Outliar Detection
- Co-location Detection

- Question Answered from Yelp Dataset
    - What type of restaurants do better in what kind of neighbourhoods ?
    - Do restaurant diversty matter in a cluster of restaurants ?

### Construting the classes of  restaurants as a response variable.
- Identifing feautures in a predictive model is half the battle. The success of the model is very much dependent on identifying good set of features. 
- How to represent success of a restaurant. Even though star rating is a good practical mesurea of selecting a restaurant for dining when using the yelp app. for prediction prupose it is not good enough baucasue it does not know how many reviews there is . so instead of star rating, review count is used. It may not be fair for restaurants started researntly so I only consider review count in last 3 years. so simply if number of reviews are high in last 3 years means restaurent is successfull. then catagoriesd in to 10 labels by decimils. 
- For demonstration purpose only Pheonix,AZ is used as a model city. The same way other cities can be analysed. 

In [110]:
#loading libraries
import json
import csv
import pandas as pd
from geopy.distance import vincenty

In [111]:
#loading review file
all_review_file = "/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.csv"
all_rw_df = pd.read_csv(all_review_file)

In [112]:
#Loading Data about all businesses
business_file ="/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json"
bf = open(business_file,'r')
#Extract Businesses in Pheonix AZ which have high customer attraction
All_biz_AZ = [json.loads(line) for line in bf if (('AZ' in json.loads(line)['state']) & ('Restaurants' not in json.loads(line)['categories']))]
All_biz_AZ = pd.DataFrame(All_biz_AZ)
print(All_biz_AZ.shape)
zip_code=[]
for i in range(All_biz_AZ.shape[0]):
    try:
        zip_code.append(All_biz_AZ['full_address'][i].split(" ")[-1])
    except:
        zip_code.append("AZ")
All_biz_AZ['zip_code'] = zip_code
#All_biz_AZ[All_biz_AZ['zip_code']=="AZ"] # Some address does not have zip code. impute those ?
#very few restaurants in this set and not many high impact businesses as well so not worth the trouble.
All_biz_AZ = All_biz_AZ[All_biz_AZ['zip_code']!="AZ"]

#remember to reset the raw index after deleting

(17245, 15)


In [113]:
#Loading Data about all restaurants
business_file ="/Users/chathura/Documents/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json"
bf = open(business_file,'r')
#Extract Businesses in Pheonix AZ which have high customer attraction
All_Res_AZ = [json.loads(line) for line in bf if (('AZ' in json.loads(line)['state']) & 
                                                  ('Restaurants'  in json.loads(line)['categories']))]
All_Res_AZ = pd.DataFrame(All_Res_AZ)
print(All_Res_AZ.shape)
zip_code=[]
for i in range(All_Res_AZ.shape[0]):
    try:
        zip_code.append(All_Res_AZ['full_address'][i].split(" ")[-1])
    except:
        zip_code.append("AZ")
All_Res_AZ['zip_code'] = zip_code
#All_biz_AZ[All_biz_AZ['zip_code']=="AZ"] # Some address does not have zip code. impute those ?
#very few restaurants in this set and not many high impact businesses as well so not worth the trouble.
All_Res_AZ = All_Res_AZ[All_Res_AZ['zip_code']!="AZ"]

(7985, 15)


In [114]:
def review_count_last3yr(res_id):
    #This function returns count of reviews happend in 2014. 
    review_count = 0
    dates = list(all_rw_df[all_rw_df['business_id'] == res_id]['date'])
    for i in dates:
        if(i.split("-")[0]=="2014" or i.split("-")[0]=="2013" or i.split("-")[0]=="2012"):
            review_count = review_count +1
    return review_count


res_ids = list(All_Res_AZ['business_id'])
rev_counts = []
for res_id in res_ids:
    rev_counts.append(review_count_last3yr(res_id))


All_Res_AZ['review_count'] = rev_counts

# Visualization of restaurants with surrounding businesses. 
The other business are catagroies in to high , medium ,low review counts. and ploted.
since there are too many businesses with less than 100 reviews.those restaures are clustered using DBScan algorithm. used cartodb to plot the businesses of the city.

In [115]:
high_Rcount_biz = All_biz_AZ[All_biz_AZ['review_count'] >= 200]
medium_Rcount_biz = All_biz_AZ[(100 < All_biz_AZ['review_count']) & (All_biz_AZ['review_count'] < 200)]
medium_Rcount_biz['R_cat']="2"
low_Rcount_biz = All_biz_AZ[All_biz_AZ['review_count'] <= 100]
high_Rcount_biz['R_cat']="3"
low_Rcount_biz['R_cat']="1"

/Users/chathura/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/chathura/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chathura/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentatio

In [116]:
#All_Res_AZ['success_cat']=pd.qcut(All_Res_AZ['rev_counts'],10,labels=["1","2","3","4","5","6","7","8","9","10"])
All_Res_AZ['success_cat']=pd.qcut(All_Res_AZ['review_count'],5,labels=["1","2","3","4","5"])
All_Res_AZ.to_csv("Arizona_restaurant_locations_with_success_cat.csv",index=False)

In [117]:
high_Rcount_biz.to_csv("Arizona_business_locations_with_high_success_cat.csv",index=False)
medium_Rcount_biz.to_csv("Arizona_business_locations_with_medium_success_cat.csv",index=False)


In [118]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
coordinates = low_Rcount_biz.as_matrix(columns=['longitude', 'latitude'])

In [119]:
db = DBSCAN(eps=.01, min_samples=3).fit(coordinates)#clusters should have 3 businessess. else is error.
labels = db.labels_
num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
clusters = pd.Series([coordinates[labels == i] for i in range(num_clusters)])
print('Number of clusters: %d' % num_clusters)

Number of clusters: 154


In [120]:
def getCentroid(points):
    n = points.shape[0]
    sum_lon = np.sum(points[:, 1])
    sum_lat = np.sum(points[:, 0])
    return (sum_lon/n, sum_lat/n)

In [121]:
def getNearestPoint(set_of_points, point_of_reference):
    closest_point = None
    closest_dist = None
    for point in set_of_points:
        point = (point[1], point[0])
        dist = great_circle(point_of_reference, point).meters
        if (closest_dist is None) or (dist < closest_dist):
            closest_point = point
            closest_dist = dist
    return closest_point

In [122]:
lon = []
lat = []
for i, cluster in clusters.iteritems():
    if len(cluster) < 3:
        representative_point = (cluster[0][1], cluster[0][0])
    else:
        representative_point = getNearestPoint(cluster, getCentroid(cluster))
    lon.append(representative_point[0])
    lat.append(representative_point[1])
rs = pd.DataFrame({'lon':lon, 'lat':lat})

In [123]:
rs['size'] = "10"
rs.to_csv("Arizona_business_locations_with_success_low_cat.csv",index=False)


In [124]:
from IPython.display import HTML # reload this with new points and success catagories.
HTML('<iframe width="100%" height="520" frameborder="0" src="https://cjgunase.cartodb.com/viz/d7f8fb6e-a5ce-11e5-8478-0e98b61680bf/embed_map" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>')

In [125]:
#by obervations, there is polarization 

Parking is a deciding factor when some one deciding to eat from a restaurnt. so having loated in a high commerial area may be an advange as well as a disadvangate so i used the avaibalitiy of paring as deciding factor for review count. 

In [126]:
# 1. building the training set from raw data by extracting subcatagories from jsan


In [127]:
#Extract Parking data from attribues and put in a seperate field
parking=[]
for i in range(All_Res_AZ.shape[0]):
    try:
        if(All_Res_AZ['attributes'][i]['Parking']['street']):
            parking.append("street")
        elif(All_Res_AZ['attributes'][i]['Parking']['lot']):
            parking.append("lot")
        elif(All_Res_AZ['attributes'][i]['Parking']['garage']):
            parking.append("garage")
        elif(All_Res_AZ['attributes'][i]['Parking']['validated']):
            parking.append("validated")
        elif(All_Res_AZ['attributes'][i]['Parking']['valet']):
            parking.append("valet")
        else:
            parking.append("street")
    except:
        parking.append("street")
        
All_Res_AZ['parking'] = parking

In [128]:
All_biz_AZ = All_biz_AZ.reset_index()# Reset the index after droping some raws missing index values give errors.
#df = df.reset_index(drop=True)
All_Res_AZ = All_Res_AZ.reset_index()

Also address can be used as a factor for patrnage. from a urban planning book i found the reason for naming roads streent avenues and differences between them in US cities. following is a difiniens of each. so it can be a desiding factor. so it is used as a catagorical variable.

In [129]:
word = ["Blvd","Blvd.","Boulevard","St"
        ,"St.","Street","Ave","Ave.","Avenue","Dr","Drive","Ln","Parkway",
        "Pkwy","Rd","Rd.","Road","Way","Wy","Wy.","Plaze","Court","Terrace","Lp.","Lp"]

In [130]:
address_type = []
for i in range(All_Res_AZ.shape[0]):
    if("St" in All_Res_AZ['full_address'][i]):
        address_type.append("Street")
    elif("Street" in All_Res_AZ['full_address'][i]):
        address_type.append("Street")
    elif("Blvd" in All_Res_AZ['full_address'][i]):
        address_type.append("Boulevard")
    elif("Boulevard" in All_Res_AZ['full_address'][i]):
        address_type.append("Boulevard")
    elif("Ave" in All_Res_AZ['full_address'][i]):
        address_type.append("Avenue")
    elif("Dr" in All_Res_AZ['full_address'][i]):
        address_type.append("Drive")
    elif("Drive" in All_Res_AZ['full_address'][i]):
        address_type.append("Drive")
    elif("Parkway" in All_Res_AZ['full_address'][i]):
        address_type.append("Parkway")
    elif("Pkwy" in All_Res_AZ['full_address'][i]):
        address_type.append("Parkway")
    elif("Rd" in All_Res_AZ['full_address'][i]):
        address_type.append("Road")
    elif("Road" in All_Res_AZ['full_address'][i]):
        address_type.append("Road")
    else:
        address_type.append("Other")

In [131]:
All_Res_AZ['address_type'] = address_type

In [132]:
res_category = []
for i in range(All_Res_AZ.shape[0]):
    if((All_Res_AZ['categories'][i][0]=='Restaurants') & (len(All_Res_AZ['categories'][i]) > 1)):
        res_category.append(All_Res_AZ['categories'][i][1])
    else:
        res_category.append(All_Res_AZ['categories'][i][0])
    
All_Res_AZ['categories'] = res_category

In [265]:
#All_Res_AZ[['business_id','latitude','longitude','categories','review_count','zip_code']]




['R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R'

In [266]:
#combine dataframs to find people attracting businessess around restaurant
df1 = All_Res_AZ[['business_id','latitude','longitude','categories','review_count','zip_code']]
df1['BizorRes']=["R" for i in range(All_Res_AZ.shape[0])]
df2 = All_biz_AZ[['business_id','latitude','longitude','categories','review_count','zip_code']]
frames = [df1, df2]
df2['BizorRes']=["B" for i in range(All_biz_AZ.shape[0])]
biz_df = pd.concat(frames)

/Users/chathura/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/chathura/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [270]:
def influence_biz(restaurant_id,biz_df):
    # This function returens businesses that are withing 2 mile radius from given restaurant.
    infbiz = [] # list of businesses around the restaurant
    row = All_Res_AZ[All_Res_AZ['business_id'] == restaurant_id]
    row1 = row
    zip_code = list(row['zip_code'])#convert from data frame object to list
    lat1 = list(row['latitude'])
    lon1 = list(row['longitude'])
    biz1 = (lat1[0],lon1[0])
    nearby = biz_df[biz_df['zip_code'] == zip_code[0]] #get all businesses near by the given restaurant
    for biz in range(nearby.shape[0]):
        near_biz_id = list(nearby[0:]['business_id'])[biz]
        if(near_biz_id == restaurant_id):
            continue
        lat2 = list(biz_df[biz_df['business_id']==near_biz_id]['latitude'])
        lon2 = list(biz_df[biz_df['business_id']==near_biz_id]['longitude'])
        biz2 = (lat2[0],lon2[0])
        if(vincenty(biz1, biz2).miles < 1):
            infbiz.append(near_biz_id)
    
    #For each of the businesses in the radius calculate the following properties.
    dep_stores = 0
    restaurents=0
    shopping = 0
    hotels = 0
    arts=0
    liesure_food = 0
    medium_size = 0
    small_size = 0
    
    for biz in infbiz:
        row = biz_df[biz_df['business_id']==biz]
        score = row['review_count'].tolist()[0]
        #print(score)
        #print(row1['categories'].tolist()[0])
        #print(row['categories'].tolist()[0])
        if("R" in row['BizorRes'].tolist()[0]):
            print (biz)
            print(score)
            restaurents=restaurents+score      
        if(score > 200):
            if("Department Stores" in row['categories'].tolist()[0]):
                dep_stores=dep_stores+score
                continue
            if("Shopping" in row['categories'].tolist()[0]):
                shopping=shopping+score
                continue
            if("Hotels" in row['categories'].tolist()[0]):
                hotels=hotels+score
                continue
            if("Arts & Entertainment" in row['categories'].tolist()[0]):
                arts=arts+score
                continue
            if("Coffee & Tea" in row['categories'].tolist()[0]):
                liesure_food=liesure_food+score
                continue
        elif(100 < score & score < 200):
            medium_size = medium_size + score
        else:
            small_size = small_size + score
        
    inf_biz_list=[dep_stores,restaurents,shopping,hotels,arts,liesure_food,medium_size,small_size]
    return inf_biz_list

In [275]:
data=[]
for i in range(5):
    print(i)
    row_list = All_Res_AZ.iloc[i].values # for efficency put the dataframe row to a list
    row = influence_biz(row_list[2],biz_df) # surrounding businesses
    row.append(row_list[7]) #lat
    row.append(row_list[8]) #lon
    row.append(row_list[3]) #resturant category
    row.append(row_list[18]) # parking
    row.append(row_list[19]) # addreesstype
    row.append(row_list[17]) #success_cat
    row.append(row_list[2])
    data.append(row)

2ZnCITVa0abGce4gZ6RhIw
3
KPoTixdjoJxSqRSEApSAGg
187
72N6TdH5MdUWE2YhYRbyxw
6
P7wKlbSRsoRrfH3DFOY08A
21
Vvh2Hd4SsZjEEfKLNKAEWw
7
sInUSy63Oh455uWs9p9OqQ
4
x8WoC_7WHzXNmt6J13Nbvw
1
OoHWabV-ym7_HWlHeXTsVA
18
oSKT5oZX1NWYaHwOWL5lag
17
QAe9UHKsxPdXQovBpaznMQ
9
q1nDSpmV-FdTBNj6UuZvTg
104
cN6aBxe2mQvrQlzk26LyRQ
232
beZVUfO1d2fS0wG8Alrvhg
0
IVc23uY-36WUNYoIbz42Fg
272
oJpmYvLibGrYPDvcaUeMOw
62
3EEVGDngt4DEhoOQZIUNUw
2
1ulwCamdqoX-SdMTcOhHDw
9
kiB6ByG9PYQLHmGpg5SGZw
1
kJFS_3WlP6TFdNUYt6V6FA
0
0dORc6ckZw5HezR3BuSsqw
259
QwaoxP5Mgm3PJuZo_4bFsw
38
Dm5K0Zvw8mqrps-jFjgIaw
20
GDmue1oJaem9ilywDtA4_A
153
fGK50PGohkCJJYamqwZ_9g
3
e3v5KRh05Qqb64U6R8U-rA
58
f1M7J3_GALq_hadsLaeGRg
7
x5Mv61CnZLohZWxfCVCPTQ
7
KPoTixdjoJxSqRSEApSAGg
187
72N6TdH5MdUWE2YhYRbyxw
6
neGAQFkBNI8-rpPBLBzNkw
3
P7wKlbSRsoRrfH3DFOY08A
21
D09yG6Z3gcsh24Qn7Y4gYA
36
kaIue7GRCmkPzDeHDBTttQ
60
VsO_rhXi5lgbaGxNwoEZsQ
0
DlCtdbceo4YNSI53cCL2lg
116
xH5axHFadmc7Veutt376TA
9
Vvh2Hd4SsZjEEfKLNKAEWw
7
sInUSy63Oh455uWs9p9OqQ
4
x8WoC_7WHzXNmt6J13Nbvw
1

In [279]:
 #create the final dataframe from the features to build a predictive model
final_df= pd.DataFrame(data, columns=['dep_stores',
                               'restaurents',
                               'shopping',
                               'hotels',
                               'arts',
                               'liesure_food',
                               'medium_business',
                               'small_business',
                               'latitude',
                               'longitude',
                               'category',
                               'parking',
                               'address_type',
                               'review_category','business_id'])

In [280]:
final_df

,dep_stores,restaurents,shopping,hotels,arts,liesure_food,medium_business,small_business,latitude,longitude,category,parking,address_type,review_category,business_id
0,0,1493,241,0,0,212,664,711,33.479542,-112.073418,Sandwiches,lot,Avenue,2,x5Mv61CnZLohZWxfCVCPTQ
1,0,5223,520,0,209,212,2602,2934,33.468988,-112.074315,American (New),street,Avenue,1,2ZnCITVa0abGce4gZ6RhIw
2,0,8019,279,0,560,0,3905,5836,33.448399,-112.071702,American (New),street,Street,3,EmzaQR5hQlF0WIl24NxAZA
3,0,1557,241,0,0,212,593,1007,33.477934,-112.073524,Thai,lot,Avenue,5,KPoTixdjoJxSqRSEApSAGg
4,0,8387,279,0,560,212,4154,6264,33.452603,-112.069119,Mexican,garage,Street,5,e5kc0CQ4R-PCCDgb274gSg


# Feature Engineering is done 